### One time run for dataset division

Here a dataset division in training-validation-test sets is applied. The results are three zip, one for each set.


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Go in the folder where the zip file of the dataset is
%cd '/content/drive/My Drive/Università/Magistrale/Vision and Perception/Project/dataset/'

In [ ]:
# Unzip the dataset (zip file) in a temporary folder on the virtual machine.
!unzip ./color.zip -d /content/localdata

In [ ]:
##### Create Train - Val - Test folders #####
# Takes the unzipped files of the dataset from the root_dir (files should be
# divided into different subfolders - one for each class) and creates a folder
# for each set in the root_root_dir containing the split part of files.
# For each class, files are randomly divided.
#####
import os
import numpy as np
import shutil
import random
root_root_dir = '/content/localdata/'
root_dir = '/content/localdata/color' 
classes_dir = []
for i in os.listdir(root_dir):
    classes_dir.append(i)

# Splitting ratios (default: 60% training, 20% validation, 20% test)
# Note that the actual division may differ a bit
val_ratio = 0.20
test_ratio = 0.20

for cls in classes_dir:
    print('/n****** CLASS ', cls, '******')
    print('> Creating class folders')
    
    os.makedirs(root_root_dir +'train/' + cls)
    os.makedirs(root_root_dir +'val/' + cls)
    os.makedirs(root_root_dir +'test/' + cls)
    

    src = root_dir + '/' + cls

    allFileNames = os.listdir(src)
    np.random.shuffle(allFileNames)
    train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                              [int(len(allFileNames)* (1 - (val_ratio + test_ratio))), 
                                                              int(len(allFileNames)* (1 - test_ratio))])


    train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
    val_FileNames = [src+'/' + name for name in val_FileNames.tolist()]
    test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]

    print('Total images in class ', cls, ': ', len(allFileNames))
    print('Training: ', len(train_FileNames))
    print('Validation: ', len(val_FileNames))
    print('Testing: ', len(test_FileNames))

    for name in train_FileNames:
        shutil.copy(name, root_root_dir +'train/' + cls)

    for name in val_FileNames:
        shutil.copy(name, root_root_dir +'val/' + cls)

    for name in test_FileNames:
        shutil.copy(name, root_root_dir +'test/' + cls)

In [ ]:
# Zip the train-val-test sets again
%cd /content/localdata
!zip -r train.zip train/
!zip -r val.zip val/
!zip -r test.zip test/

In [ ]:
# Copy the three zip files on Drive
!cp train.zip '/content/drive/My Drive/Università/Magistrale/Vision and Perception/Project/dataset/train.zip'
!cp val.zip '/content/drive/My Drive/Università/Magistrale/Vision and Perception/Project/dataset/val.zip'
!cp test.zip '/content/drive/My Drive/Università/Magistrale/Vision and Perception/Project/dataset/test.zip'

Here we create the zip of our second test set

In [ ]:
!zip -r simili_dataset_NEW.zip 'Nuove_simili_al_dataset/'

### Prepare data for training

To speed up the training phase, the data set is loaded locally on the Colab virtual machine.
So here the train, val and test zips are unzipped so to have all files available locally.
[To do just once when the VM is started]

In [ ]:
# Go to the folder where the zips are
%cd '/content/drive/My Drive/Università/Magistrale/Vision and Perception/Project/dataset/'

/content/drive/My Drive/Università/Magistrale/Vision and Perception/Project/dataset


In [ ]:
# Just a few seconds and all files will be on the content path of the Colab VM
!unzip ./train.zip -d /content/localdata
!unzip ./val.zip -d /content/localdata
!unzip ./test.zip -d /content/localdata

In [ ]:
# This does the same for the data set we built
!unzip '/content/drive/My Drive/Università/Magistrale/Vision and Perception/Project/dataset/simili_dataset_NEW.zip' -d /content/localdata/

### Data Augmentation

Data Augmentation is essential to improve results and reduce overfitting.

Here we made experiments with different techniques:


*   Horizontal Flip
*   Vertical Flip
*   Random crop
*   Rotation range
*   Brightness range

In the final presentation we will show our considerations on the results we got during tests.


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os, datetime
from tensorflow import keras
import time
from tensorflow.keras import models, layers, optimizers, applications
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from skimage.transform import resize

In [ ]:
# Go to the project folder on Drive
%cd '/content/drive/My Drive/Università/Magistrale/Vision and Perception/Project/'

In [ ]:
# Set the path for all the 
train_data_dir = '/content/localdata/train/'
val_data_dir = '/content/localdata/val/'
test_data_dir = '/content/localdata/test/'
target_image_size = (256,256)

In [ ]:
# Random crop is not natively implemented in the ImageDataGenerator class, so we
# made our version
def random_crop(image):
    height, width = image.shape[:2]
    random_array = np.random.random(size=3);
    delta = int((width*0.5) * (1+random_array[0]*0.5))
    x = int(random_array[1] * (width-delta))
    y = int(random_array[2] * (height-delta))

    image_crop = image[y:y+delta, x:x+delta, 0:3]
    image_crop_resize = resize(image_crop, image.shape)
    return image_crop_resize


In [ ]:
# Activate/Deactive the desidered data aug techniques
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                  #horizontal_flip=True,
                                                                  #brightness_range=[0.5,1.5],
                                                                  #vertical_flip=True,
                                                                  #rotation_range=10,
                                                                  #preprocessing_function=random_crop
                                                                  )
# No data augmentation is applied to the test set
test_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:
train_data_gen = image_generator.flow_from_directory(
                                                     directory=train_data_dir,
                                                     color_mode='rgb',
                                                     batch_size=32,
                                                     shuffle=True,
                                                     seed=123,
                                                     target_size=target_image_size
                                                     )

In [ ]:
val_data_gen = image_generator.flow_from_directory(
                                                    directory=val_data_dir,
                                                    batch_size=32,
                                                    color_mode='rgb',
                                                    shuffle=True,
                                                    seed=123,
                                                    target_size=target_image_size)

In [ ]:
test_data_gen = test_image_generator.flow_from_directory(
                                                    directory=test_data_dir,
                                                    batch_size=32,
                                                    shuffle=False,
                                                    target_size=target_image_size)

In [ ]:
# We also create two simple dict with class name <-> index class
classes_dict = train_data_gen.class_indices
to_classes_dict = {index: classe for classe, index in classes_dict.items()}

In [ ]:
# Here there's a different dict where each class is associated to the image count
# of that class (used to print the following histogram)
classes = {}
for i in os.listdir('./color/'):
  path = './color/' + i + '/'
  count = 0
  for f in os.listdir(path):
    if os.path.isfile(os.path.join(path, f)):
        count += 1
  classes[i] = count
classes

In [ ]:
# Generate the histogram of the class distribution
plt.figure(figsize=(15,10))
x = [elem for elem in classes.keys()]
y = [n for n in classes.values()]
plt.gcf().subplots_adjust(bottom=0.42)
plt.bar(x, y, color='b')
plt.xticks(x, classes.keys(), rotation='vertical')
plt.savefig('/content/drive/My Drive/Università/Magistrale/Vision and Perception/Project/dataset/images_distribution.jpg')

### Model definition: our base model



In [ ]:
# To set the learning rate
optimizer = keras.optimizers.Adam(lr=0.0001)

In [ ]:
def build_model():
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(256, 256, 3)))
  model.add(layers.MaxPooling2D())
  model.add(layers.BatchNormalization())

  model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
  model.add(layers.MaxPooling2D())
  model.add(layers.BatchNormalization())

  model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
  model.add(layers.MaxPooling2D())
  model.add(layers.BatchNormalization())

  model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
  model.add(layers.MaxPooling2D())
  model.add(layers.BatchNormalization())

  model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
  model.add(layers.MaxPooling2D())
  model.add(layers.BatchNormalization())
  
  model.add(layers.Flatten())


  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(38, activation='softmax'))
  
  model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                #optimizer = optimizer,
                metrics=['accuracy'])
  return model

In [ ]:
model = build_model()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 32)      128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        3

### Model definition: Pretrained model

We made tests with VGG, ResNet, Inception and DenseNet implementing both feature extraction and fine tuning.

In [ ]:
resnet = tf.keras.applications.ResNet50V2(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
resnet.summary()

In [ ]:
# To freeze a subset of layers: resnet.layers[:-X]
for layer in resnet.layers:
  layer.trainable = False

In [ ]:
def build_model():
  model = models.Sequential()
  model.add(resnet)
  model.add(layers.GlobalAveragePooling2D())
  model.add(layers.Dense(1024, activation='relu'))
  model.add(layers.Dense(38, activation='softmax'))

  model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  
  return model

In [ ]:
model = build_model()

model.summary()

### Tensorboard

In [ ]:
overwrite_log_dir = ""

root = os.getcwd()
if (overwrite_log_dir==""):
  log_dir = os.path.join(root, "logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
else: 
  log_dir = overwrite_log_dir
log_dir

'/content/drive/My Drive/Università/Magistrale/Vision and Perception/Project/logs/20200919-124334'

In [ ]:
%load_ext tensorboard

In [ ]:
# ONLY IF NEEDED!
!rm -rf '/content/drive/My Drive/Università/Vision and Perception/Project/logs'

In [ ]:
# It should be used (changing the pid) to kill the Tensorflow process when it
# starts to throw a tantrum. Though, rarely killed the process
!kill 11324

In [ ]:
%tensorboard --logdir logs

### Training

In [ ]:
# FE = Feature Extraction
# FT = Fine Tuning
# HF = Horizontal flip
# RR = Rotation range
# RC = Random cropping
# RB = Random brightness
# GAP = Global Average Pooling
# FL = Flatten
# Dx = Dense layer with x units (i.e. D512)

# Name template: test{X}_{nameModel}_{typeTraining}_{addedLayers}_{dataAugTechniques}.h5

path_model = '/content/drive/My Drive/Università/Magistrale/Vision and Perception/Project/models/test123_ourmodel_RB_RR_HF.h5'

callbacks = [
    TensorBoard(log_dir, update_freq = 50),
    EarlyStopping(monitor='val_accuracy', patience=5),
    ModelCheckpoint(path_model, save_best_only=True)
]

In [ ]:
# Set class weights used for weighting the loss function, thus keeping into account
# the imbalance of the data set.
# Here the output is a dict passed when training the model
class_weights = class_weight.compute_class_weight('balanced',
                                                 list(to_classes_dict.keys()),
                                                 train_data_gen.classes.tolist())
dict_weights ={}
for i in range(len(class_weights)):
  dict_weights[i] = class_weights[i]
print(dict_weights)

In [ ]:
history = model.fit(train_data_gen, epochs=40,
                    validation_data=val_data_gen,
                    initial_epoch=0,
                    shuffle=True,
                    max_queue_size = 256,
                    callbacks=callbacks,
                    #class_weight=class_weights)

In [ ]:
# accuracy plot
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('test_123_plot_accuracy')
plt.show()


# loss plot
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('test_123_plot_loss')
plt.show()

### Test phase

In this part of the notebook we made several tests on both the test set of the main data set and on an hand-made test set*.

*We visited some friends, relatives and local farmers and collected more than 350 images of leaves. Of these photos we selected ~200 images to form our test set.

#### Only for the hand-made test set

In [ ]:
folder_test_set = 'Nuove_simili_al_dataset'

In [ ]:
for folder in classes_dict.keys():
  if (not os.path.isdir( '/content/localdata/' + folder_test_set +'/' + folder)):
    os.makedirs('/content/localdata/' + folder_test_set +'/' + folder)

In [ ]:
test_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_data_gen = test_image_generator.flow_from_directory(
                                                    directory='/content/localdata/' + folder_test_set +'/',
                                                    batch_size=5,
                                                    target_size=(256,256),
                                                    shuffle=False)

#### Evaluate, Confusion Matrix and Classification Report

In [ ]:
model.evaluate(test_data_gen)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sn

In [ ]:
y_pred_all = model.predict(test_data_gen)
y_pred = np.argmax(y_pred_all, axis=-1)

In [ ]:
print('Confusion Matrix')
conf_matrix=confusion_matrix(test_data_gen.classes, y_pred)
conf_matrix = tf.math.confusion_matrix(test_data_gen.classes, y_pred)
plt.figure(figsize = (20,12))
sn.heatmap(conf_matrix, annot=True, fmt='.0f', cmap='coolwarm', xticklabels=classes_dict.keys(), yticklabels=classes_dict.keys())
print('Classification Report')
print(classification_report(test_data_gen.classes, y_pred, zero_division=0))

In [ ]:
# This module here is used to get a colorful version of the classification report.
# It is not really essential, but it helps in identifying critical values instantly
# The code has been taken from Stack Overflow. We left all the credits there.

def show_values(pc, fmt="%.2f", **kw):
    '''
    Heatmap with text in each cell with matplotlib's pyplot
    Source: https://stackoverflow.com/a/25074150/395857 
    By HYRY
    '''
    import itertools
    pc.update_scalarmappable()
    ax = pc.axes
    for p, color, value in zip(pc.get_paths(), pc.get_facecolors(), pc.get_array()):
        x, y = p.vertices[:-2, :].mean(0)
        if np.all(color[:3] > 0.5):
            color = (0.0, 0.0, 0.0)
        else:
            color = (1.0, 1.0, 1.0)
        ax.text(x, y, fmt % value, ha="center", va="center", color=color, **kw)


def cm2inch(*tupl):
    '''
    Specify figure size in centimeter in matplotlib
    Source: https://stackoverflow.com/a/22787457/395857
    By gns-ank
    '''
    inch = 2.54
    if type(tupl[0]) == tuple:
        return tuple(i/inch for i in tupl[0])
    else:
        return tuple(i/inch for i in tupl)


def heatmap(AUC, title, xlabel, ylabel, xticklabels, yticklabels, figure_width=40, figure_height=20, correct_orientation=False, cmap='RdBu'):
    '''
    Inspired by:
    - https://stackoverflow.com/a/16124677/395857 
    - https://stackoverflow.com/a/25074150/395857
    '''

    # Plot it out
    fig, ax = plt.subplots()    
    #c = ax.pcolor(AUC, edgecolors='k', linestyle= 'dashed', linewidths=0.2, cmap='RdBu', vmin=0.0, vmax=1.0)
    c = ax.pcolor(AUC, edgecolors='k', linestyle= 'dashed', linewidths=0.2, cmap=cmap)

    # put the major ticks at the middle of each cell
    ax.set_yticks(np.arange(AUC.shape[0]) + 0.5, minor=False)
    ax.set_xticks(np.arange(AUC.shape[1]) + 0.5, minor=False)

    # set tick labels
    #ax.set_xticklabels(np.arange(1,AUC.shape[1]+1), minor=False)
    ax.set_xticklabels(xticklabels, minor=False)
    ax.set_yticklabels(yticklabels, minor=False)

    # set title and x/y labels
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)      

    # Remove last blank column
    plt.xlim( (0, AUC.shape[1]) )

    # Turn off all the ticks
    ax = plt.gca()    
    for t in ax.xaxis.get_major_ticks():
        t.tick1On = False
        t.tick2On = False
    for t in ax.yaxis.get_major_ticks():
        t.tick1On = False
        t.tick2On = False

    # Add color bar
    plt.colorbar(c)

    # Add text in each cell 
    show_values(c)

    # Proper orientation (origin at the top left instead of bottom left)
    if correct_orientation:
        ax.invert_yaxis()
        ax.xaxis.tick_top()       

    # resize 
    fig = plt.gcf()
    #fig.set_size_inches(cm2inch(40, 20))
    #fig.set_size_inches(cm2inch(40*4, 20*4))
    fig.set_size_inches(cm2inch(figure_width, figure_height))



def plot_classification_report(classification_report, title='Classification report ', cmap='RdBu'):
    '''
    Plot scikit-learn classification report.
    Extension based on https://stackoverflow.com/a/31689645/395857 
    '''
    lines = classification_report.split('\n')

    classes = []
    plotMat = []
    support = []
    class_names = []
    for line in lines[2 : (len(lines) - 4)]:
        t = line.strip().split()
        if len(t) < 2: continue
        classes.append(t[0])
        v = [float(x) for x in t[1: len(t) - 1]]
        support.append(int(t[-1]))
        class_names.append(to_classes_dict[int(t[0])])
        print(v)
        plotMat.append(v)

    print('plotMat: {0}'.format(plotMat))
    print('support: {0}'.format(support))

    xlabel = 'Metrics'
    ylabel = 'Classes'
    xticklabels = ['Precision', 'Recall', 'F1-score']
    yticklabels = ['{0} ({1})'.format(class_names[idx], sup) for idx, sup  in enumerate(support)]
    figure_width = 25
    figure_height = len(class_names) + 7
    correct_orientation = False
    heatmap(np.array(plotMat), title, xlabel, ylabel, xticklabels, yticklabels, figure_width, figure_height, correct_orientation, cmap=cmap)


In [ ]:
plot_classification_report(classification_report(test_data_gen.classes, y_pred), "Classification Report")

Some small tests

In [ ]:
# Pick one image and make the predict on it
image = keras.preprocessing.image.load_img('/content/drive/My Drive/Università/Magistrale/Vision and Perception/Project/dataset2/squash_test.jpg')


In [ ]:
image = resize(np.array(image), (256,256))
image = image2.reshape((1,256,256,3))
image.shape

In [ ]:
predictions = model.predict(image, batch_size = 1)
to_classes_dict[np.argmax(predictions)]

Load or save a model

In [ ]:
# Save a model
keras.models.save_model(model,'/content/drive/My Drive/Università/Magistrale/Vision and Perception/Project/models/nostro_test88_fine.h5')

In [ ]:
# Load a previously trained model
model = keras.models.load_model('/content/drive/My Drive/Università/Magistrale/Vision and Perception/Project/models/test2_DenseNet201_FT_GAP_D1024_HF_RR_RC.h5')

In [ ]:
model.summary()